In [1]:
import numpy as np

# Load feature files
X1_train = np.load(r"D:\Tsetlin Machines\REDRESS Training\CIFAR2\train\3X3_train.npy")
X1_test = np.load(r"D:\Tsetlin Machines\REDRESS Training\CIFAR2\test\3X3_test.npy")

X2_train = np.load(r"D:\Tsetlin Machines\REDRESS Training\CIFAR2\train\4X4_train.npy")
X2_test = np.load(r"D:\Tsetlin Machines\REDRESS Training\CIFAR2\test\4X4_test.npy")

X3_train= np.load(r"D:\Tsetlin Machines\REDRESS Training\CIFAR2\train\HOG_train.npy")
X3_test= np.load(r"D:\Tsetlin Machines\REDRESS Training\CIFAR2\test\HOG_test.npy")

X4_train = np.load(r"D:\Tsetlin Machines\REDRESS Training\CIFAR2\train\Thresh_train.npy")
X4_test= np.load(r"D:\Tsetlin Machines\REDRESS Training\CIFAR2\test\Thresh_test.npy")


# Load labels
y_train = np.load(r"D:\Tsetlin Machines\REDRESS Training\CIFAR2\labels\train_labels.npy")
y_test = np.load(r"D:\Tsetlin Machines\REDRESS Training\CIFAR2\labels\test_labels.npy")


In [2]:
np.unique(y_train)

array([0, 1])

In [3]:
import pandas as pd
from sklearn.preprocessing import KBinsDiscretizer
def quantile_binning(data, n_bins=4):
    
    binned_data = []
    for column in data.columns:
        discretizer = KBinsDiscretizer(n_bins=n_bins, encode='ordinal', strategy='quantile')
        column_binned = discretizer.fit_transform(data[[column]]).astype(int)
        binned_data.append(column_binned)

    binned_data = np.hstack(binned_data)
    return pd.DataFrame(binned_data, columns=data.columns)

def booleanize(binned_data):
    
    booleanized_data = []
    column_names = []

    for column in binned_data.columns:
        unique_bins = sorted(binned_data[column].unique())  
        for bin_value in unique_bins:
            boolean_column = (binned_data[column] == bin_value).astype(int)
            booleanized_data.append(boolean_column)
            column_names.append(f"{column}_bin_{bin_value}")
    

    booleanized_data = np.column_stack(booleanized_data)
    return pd.DataFrame(booleanized_data, columns=column_names)


def booleanize_automated(num_array,binn):
    num_array=pd.DataFrame(num_array)
    num_bin=quantile_binning(num_array,binn)
    num_final=booleanize(num_bin)
    return num_final


In [4]:
features1 = {
    "X1_train": X1_train, "X1_test": X1_test,
    "X2_train": X2_train, "X2_test": X2_test,
    "X3_train": X3_train, "X3_test": X3_test,
    "X4_train": X4_train, "X4_test": X4_test,
}

def booleanize_all(features_dict, skip_keys=None, binn=5):
    skip_keys = skip_keys or []
    for key in features_dict:
        if key in skip_keys:
            if hasattr(features_dict[key], "to_numpy"):
                features_dict[key] = features_dict[key].to_numpy()
        else:
            features_dict[key] = booleanize_automated(features_dict[key], binn=binn)
            if hasattr(features_dict[key], "to_numpy"):
                features_dict[key] = features_dict[key].to_numpy()
    return features_dict

features = booleanize_all(features1, skip_keys=["X4_train", "X4_test"], binn=5)

d:\Anaconda Projects\env\Lib\site-packages\sklearn\preprocessing\_discretization.py:296: FutureWarning: The current default behavior, quantile_method='linear', will be changed to quantile_method='averaged_inverted_cdf' in scikit-learn version 1.9 to naturally support sample weight equivalence properties by default. Pass quantile_method='averaged_inverted_cdf' explicitly to silence this warning.
  warnings.warn(
d:\Anaconda Projects\env\Lib\site-packages\sklearn\preprocessing\_discretization.py:296: FutureWarning: The current default behavior, quantile_method='linear', will be changed to quantile_method='averaged_inverted_cdf' in scikit-learn version 1.9 to naturally support sample weight equivalence properties by default. Pass quantile_method='averaged_inverted_cdf' explicitly to silence this warning.
  warnings.warn(
d:\Anaconda Projects\env\Lib\site-packages\sklearn\preprocessing\_discretization.py:296: FutureWarning: The current default behavior, quantile_method='linear', will be ch

In [33]:
y_train

array([0, 1, 1, ..., 1, 1, 1], shape=(50000,))

In [5]:
features["X4_test"].shape

(10000, 1025)

In [6]:
features["X3_test"].shape

(10000, 1620)

In [7]:
features["X2_test"].shape

(10000, 960)

In [8]:
features["X1_test"].shape

(10000, 1815)

In [26]:
def save_data_to_txt(X_bool: np.ndarray, y: np.ndarray, filename: str):
    with open(filename, 'w') as f:
        for i in range(len(y)):
            line = ' '.join(map(str, X_bool[i])) + f' {y[i]}\n'
            f.write(line)

# Example usage:
save_data_to_txt(features["X1_train"], y_train, '3x3_train.txt')
save_data_to_txt(features["X2_train"], y_train, '4x4_train.txt')
save_data_to_txt(features["X3_train"], y_train, 'hog_train.txt')


# TM Training

# X1

In [18]:
import ctypes
import numpy as np

feature=features["X1_train"]
labels = np.asarray(y_train)

NUM_CLASSES = 2
FEATURES = 1815

NUM_EPOCHS = 50
s = 7.5  #float


# Load the shared library
lib = ctypes.CDLL('./DLLs/tm_cifar2_3x3.dll')  # Adjust path if needed

lib.CreateTsetlinMachine.restype = ctypes.c_void_p
lib.CreateTsetlinMachine.argtypes = []

# tm_update(TM_ptr, Xi_ptr, target, s)
lib.tm_update.restype = None
lib.tm_update.argtypes = [
    ctypes.c_void_p,
    ctypes.POINTER(ctypes.c_int),
    ctypes.c_int,
    ctypes.c_float
]

# tm_score(TM_ptr, Xi_ptr)
lib.tm_score.restype = ctypes.c_int
lib.tm_score.argtypes = [
    ctypes.c_void_p,
    ctypes.POINTER(ctypes.c_int)
]

# Optional: expose additional utilities
lib.tm_show_value.restype = ctypes.c_uint
lib.tm_show_value.argtypes = [ctypes.c_void_p, ctypes.c_int]

lib.tm_get_state.restype = ctypes.c_int
lib.tm_get_state.argtypes = [ctypes.c_void_p, ctypes.c_int, ctypes.c_int, ctypes.c_int]

# === Initialize Tsetlin Machines ===
# One binary TM per class for multiclass classification
tmachines = [lib.CreateTsetlinMachine() for _ in range(NUM_CLASSES)]

# === Training Loop ===
for epoch in range(NUM_EPOCHS):
    print(f"Epoch {epoch+1}/{NUM_EPOCHS}")
    for Xi_row, true_label in zip(feature, labels):
        # Convert feature vector to ctypes array
        Xi_ctypes = (ctypes.c_int * FEATURES)(*Xi_row.tolist())
        # Update each class-specific TM
        for cls in range(NUM_CLASSES):
            target = 1 if cls == true_label else 0
            lib.tm_update(tmachines[cls], Xi_ctypes, target, ctypes.c_float(s))
    print("Epoch complete.")

# === Prediction on a sample ===
sample_idx = 0  # adjust as needed
Xi_test = (ctypes.c_int * FEATURES)(*feature[sample_idx].tolist())
class_sums = [lib.tm_score(tm, Xi_test) for tm in tmachines]
predicted_class = int(np.argmax(class_sums))

print("Class vote sums:")
for cls, vote in enumerate(class_sums):
    print(f"  Class {cls}: {vote}")
print(f"Predicted class for sample {sample_idx}: {predicted_class}")


Epoch 1/50
Epoch complete.
Epoch 2/50
Epoch complete.
Epoch 3/50
Epoch complete.
Epoch 4/50
Epoch complete.
Epoch 5/50
Epoch complete.
Epoch 6/50
Epoch complete.
Epoch 7/50
Epoch complete.
Epoch 8/50
Epoch complete.
Epoch 9/50
Epoch complete.
Epoch 10/50
Epoch complete.
Epoch 11/50
Epoch complete.
Epoch 12/50
Epoch complete.
Epoch 13/50
Epoch complete.
Epoch 14/50
Epoch complete.
Epoch 15/50
Epoch complete.
Epoch 16/50
Epoch complete.
Epoch 17/50
Epoch complete.
Epoch 18/50
Epoch complete.
Epoch 19/50
Epoch complete.
Epoch 20/50
Epoch complete.
Epoch 21/50
Epoch complete.
Epoch 22/50
Epoch complete.
Epoch 23/50
Epoch complete.
Epoch 24/50
Epoch complete.
Epoch 25/50
Epoch complete.
Epoch 26/50
Epoch complete.
Epoch 27/50
Epoch complete.
Epoch 28/50
Epoch complete.
Epoch 29/50
Epoch complete.
Epoch 30/50
Epoch complete.
Epoch 31/50
Epoch complete.
Epoch 32/50
Epoch complete.
Epoch 33/50
Epoch complete.
Epoch 34/50
Epoch complete.
Epoch 35/50
Epoch complete.
Epoch 36/50
Epoch complete.
E

In [19]:
results = []
X1_test=features["X1_test"]
for i in range(X1_test.shape[0]):
    Xi_test = (ctypes.c_int * FEATURES)(*X1_test[i])
    class_sums = [lib.tm_score(tm, Xi_test) for tm in tmachines]
    predicted_label = int(np.argmax(class_sums))
    results.append([i] + class_sums + [predicted_label])

In [20]:
# Define CSV filename.
import csv
csv_filename = "cifar2_3x3_C200_CL2_T30_E50.csv"

# Write the results to a CSV file.
with open(csv_filename, "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    # Write header: sample index, one column per class vote sum, and predicted label.
    header = ["sample_index"] + [f"class{cls}_sum" for cls in range(NUM_CLASSES)] + ["predicted_label"]
    writer.writerow(header)
    writer.writerows(results)

print(f"Predictions saved to {csv_filename}")

Predictions saved to cifar2_3x3_C200_CL2_T30_E50.csv


In [21]:
import pandas as pd
import numpy as np

csv_files = [
    "cifar2_3x3_C200_CL2_T30_E50.csv",
    # "cifar2_4x4_C200_CL2_T30_E50.csv",
    # "cifar2_hog_C200_CL2_T30_E50.csv",
    # "cifar2_thresh_C200_CL2_T30_E50.csv"
]


dfs = [pd.read_csv(f) for f in csv_files]



class_sums = [df[[f"class{i}_sum" for i in range(2)]].values for df in dfs]


total_class_votes = np.sum(class_sums, axis=0)


overall_preds = np.argmax(total_class_votes, axis=1)


accuracy = np.mean(overall_preds == y_test)
print(f"Overall Accuracy: {accuracy * 100:.2f}%")

Overall Accuracy: 84.55%


In [22]:
lib.tm_save.argtypes = [ctypes.c_void_p, ctypes.c_char_p]
lib.tm_save.restype  = ctypes.c_int

# # Save each TM to its own file:

# for cls, tm in enumerate(tmachines):
#     fname = f"tm_class{cls}.bin".encode('ascii')
#     ret = lib.tm_save(tm, fname)
#     if ret != 0:
#         raise RuntimeError(f"tm_save failed for class {cls} (err {ret})")
# print("All TMs saved.")

# Save each TM to a directory:
save_dir=r"D:\Tsetlin Machines\REDRESS Training\CIFAR2 Model\3x3"
import os
os.makedirs(save_dir, exist_ok=True)

for cls, tm in enumerate(tmachines):
    fname = os.path.join(save_dir, f"tm_class{cls}.bin").encode('ascii')
    ret = lib.tm_save(tm, fname)
    if ret != 0:
        raise RuntimeError(f"tm_save failed for class {cls} (err {ret})")

# X2

In [23]:
import ctypes
import numpy as np

feature=features["X2_train"]
labels = np.asarray(y_train)

NUM_CLASSES = 2
FEATURES = 960

NUM_EPOCHS = 50
s = 7.5  #float


# Load the shared library
lib = ctypes.CDLL('./DLLs/tm_cifar2_4x4.dll')  # Adjust path if needed

lib.CreateTsetlinMachine.restype = ctypes.c_void_p
lib.CreateTsetlinMachine.argtypes = []

# tm_update(TM_ptr, Xi_ptr, target, s)
lib.tm_update.restype = None
lib.tm_update.argtypes = [
    ctypes.c_void_p,
    ctypes.POINTER(ctypes.c_int),
    ctypes.c_int,
    ctypes.c_float
]

# tm_score(TM_ptr, Xi_ptr)
lib.tm_score.restype = ctypes.c_int
lib.tm_score.argtypes = [
    ctypes.c_void_p,
    ctypes.POINTER(ctypes.c_int)
]

# Optional: expose additional utilities
lib.tm_show_value.restype = ctypes.c_uint
lib.tm_show_value.argtypes = [ctypes.c_void_p, ctypes.c_int]

lib.tm_get_state.restype = ctypes.c_int
lib.tm_get_state.argtypes = [ctypes.c_void_p, ctypes.c_int, ctypes.c_int, ctypes.c_int]

# === Initialize Tsetlin Machines ===
# One binary TM per class for multiclass classification
tmachines = [lib.CreateTsetlinMachine() for _ in range(NUM_CLASSES)]

# === Training Loop ===
for epoch in range(NUM_EPOCHS):
    print(f"Epoch {epoch+1}/{NUM_EPOCHS}")
    for Xi_row, true_label in zip(feature, labels):
        # Convert feature vector to ctypes array
        Xi_ctypes = (ctypes.c_int * FEATURES)(*Xi_row.tolist())
        # Update each class-specific TM
        for cls in range(NUM_CLASSES):
            target = 1 if cls == true_label else 0
            lib.tm_update(tmachines[cls], Xi_ctypes, target, ctypes.c_float(s))
    print("Epoch complete.")

# === Prediction on a sample ===
sample_idx = 0  # adjust as needed
Xi_test = (ctypes.c_int * FEATURES)(*feature[sample_idx].tolist())
class_sums = [lib.tm_score(tm, Xi_test) for tm in tmachines]
predicted_class = int(np.argmax(class_sums))

print("Class vote sums:")
for cls, vote in enumerate(class_sums):
    print(f"  Class {cls}: {vote}")
print(f"Predicted class for sample {sample_idx}: {predicted_class}")


Epoch 1/50
Epoch complete.
Epoch 2/50
Epoch complete.
Epoch 3/50
Epoch complete.
Epoch 4/50
Epoch complete.
Epoch 5/50
Epoch complete.
Epoch 6/50
Epoch complete.
Epoch 7/50
Epoch complete.
Epoch 8/50
Epoch complete.
Epoch 9/50
Epoch complete.
Epoch 10/50
Epoch complete.
Epoch 11/50
Epoch complete.
Epoch 12/50
Epoch complete.
Epoch 13/50
Epoch complete.
Epoch 14/50
Epoch complete.
Epoch 15/50
Epoch complete.
Epoch 16/50
Epoch complete.
Epoch 17/50
Epoch complete.
Epoch 18/50
Epoch complete.
Epoch 19/50
Epoch complete.
Epoch 20/50
Epoch complete.
Epoch 21/50
Epoch complete.
Epoch 22/50
Epoch complete.
Epoch 23/50
Epoch complete.
Epoch 24/50
Epoch complete.
Epoch 25/50
Epoch complete.
Epoch 26/50
Epoch complete.
Epoch 27/50
Epoch complete.
Epoch 28/50
Epoch complete.
Epoch 29/50
Epoch complete.
Epoch 30/50
Epoch complete.
Epoch 31/50
Epoch complete.
Epoch 32/50
Epoch complete.
Epoch 33/50
Epoch complete.
Epoch 34/50
Epoch complete.
Epoch 35/50
Epoch complete.
Epoch 36/50
Epoch complete.
E

In [24]:
results = []
X1_test=features["X2_test"]
for i in range(X1_test.shape[0]):
    Xi_test = (ctypes.c_int * FEATURES)(*X1_test[i])
    class_sums = [lib.tm_score(tm, Xi_test) for tm in tmachines]
    predicted_label = int(np.argmax(class_sums))
    results.append([i] + class_sums + [predicted_label])

In [25]:
# Define CSV filename.
import csv
csv_filename = "cifar2_4x4_C200_CL2_T30_E50.csv"

# Write the results to a CSV file.
with open(csv_filename, "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    # Write header: sample index, one column per class vote sum, and predicted label.
    header = ["sample_index"] + [f"class{cls}_sum" for cls in range(NUM_CLASSES)] + ["predicted_label"]
    writer.writerow(header)
    writer.writerows(results)

print(f"Predictions saved to {csv_filename}")

Predictions saved to cifar2_4x4_C200_CL2_T30_E50.csv


In [26]:
import pandas as pd
import numpy as np

csv_files = [
    # "cifar2_3x3_C200_CL2_T30_E50.csv",
    "cifar2_4x4_C200_CL2_T30_E50.csv",
    # "cifar2_hog_C200_CL2_T30_E50.csv",
    # "cifar2_thresh_C200_CL2_T30_E50.csv"
]


dfs = [pd.read_csv(f) for f in csv_files]



class_sums = [df[[f"class{i}_sum" for i in range(2)]].values for df in dfs]


total_class_votes = np.sum(class_sums, axis=0)


overall_preds = np.argmax(total_class_votes, axis=1)


accuracy = np.mean(overall_preds == y_test)
print(f"Overall Accuracy: {accuracy * 100:.2f}%")

Overall Accuracy: 84.16%


In [27]:
lib.tm_save.argtypes = [ctypes.c_void_p, ctypes.c_char_p]
lib.tm_save.restype  = ctypes.c_int

# # Save each TM to its own file:

# for cls, tm in enumerate(tmachines):
#     fname = f"tm_class{cls}.bin".encode('ascii')
#     ret = lib.tm_save(tm, fname)
#     if ret != 0:
#         raise RuntimeError(f"tm_save failed for class {cls} (err {ret})")
# print("All TMs saved.")

# Save each TM to a directory:
save_dir=r"D:\Tsetlin Machines\REDRESS Training\CIFAR2 Model\4x4"
import os
os.makedirs(save_dir, exist_ok=True)

for cls, tm in enumerate(tmachines):
    fname = os.path.join(save_dir, f"tm_class{cls}.bin").encode('ascii')
    ret = lib.tm_save(tm, fname)
    if ret != 0:
        raise RuntimeError(f"tm_save failed for class {cls} (err {ret})")

# X3

In [28]:
import ctypes
import numpy as np

feature=features["X3_train"]
labels = np.asarray(y_train)

NUM_CLASSES = 2
FEATURES = 1620

NUM_EPOCHS = 50
s = 7.5  #float


# Load the shared library
lib = ctypes.CDLL('./DLLs/tm_cifar2_hog.dll')  # Adjust path if needed

lib.CreateTsetlinMachine.restype = ctypes.c_void_p
lib.CreateTsetlinMachine.argtypes = []

# tm_update(TM_ptr, Xi_ptr, target, s)
lib.tm_update.restype = None
lib.tm_update.argtypes = [
    ctypes.c_void_p,
    ctypes.POINTER(ctypes.c_int),
    ctypes.c_int,
    ctypes.c_float
]

# tm_score(TM_ptr, Xi_ptr)
lib.tm_score.restype = ctypes.c_int
lib.tm_score.argtypes = [
    ctypes.c_void_p,
    ctypes.POINTER(ctypes.c_int)
]

# Optional: expose additional utilities
lib.tm_show_value.restype = ctypes.c_uint
lib.tm_show_value.argtypes = [ctypes.c_void_p, ctypes.c_int]

lib.tm_get_state.restype = ctypes.c_int
lib.tm_get_state.argtypes = [ctypes.c_void_p, ctypes.c_int, ctypes.c_int, ctypes.c_int]

# === Initialize Tsetlin Machines ===
# One binary TM per class for multiclass classification
tmachines = [lib.CreateTsetlinMachine() for _ in range(NUM_CLASSES)]

# === Training Loop ===
for epoch in range(NUM_EPOCHS):
    print(f"Epoch {epoch+1}/{NUM_EPOCHS}")
    for Xi_row, true_label in zip(feature, labels):
        # Convert feature vector to ctypes array
        Xi_ctypes = (ctypes.c_int * FEATURES)(*Xi_row.tolist())
        # Update each class-specific TM
        for cls in range(NUM_CLASSES):
            target = 1 if cls == true_label else 0
            lib.tm_update(tmachines[cls], Xi_ctypes, target, ctypes.c_float(s))
    print("Epoch complete.")

# === Prediction on a sample ===
sample_idx = 0  # adjust as needed
Xi_test = (ctypes.c_int * FEATURES)(*feature[sample_idx].tolist())
class_sums = [lib.tm_score(tm, Xi_test) for tm in tmachines]
predicted_class = int(np.argmax(class_sums))

print("Class vote sums:")
for cls, vote in enumerate(class_sums):
    print(f"  Class {cls}: {vote}")
print(f"Predicted class for sample {sample_idx}: {predicted_class}")


Epoch 1/50
Epoch complete.
Epoch 2/50
Epoch complete.
Epoch 3/50
Epoch complete.
Epoch 4/50
Epoch complete.
Epoch 5/50
Epoch complete.
Epoch 6/50
Epoch complete.
Epoch 7/50
Epoch complete.
Epoch 8/50
Epoch complete.
Epoch 9/50
Epoch complete.
Epoch 10/50
Epoch complete.
Epoch 11/50
Epoch complete.
Epoch 12/50
Epoch complete.
Epoch 13/50
Epoch complete.
Epoch 14/50
Epoch complete.
Epoch 15/50
Epoch complete.
Epoch 16/50
Epoch complete.
Epoch 17/50
Epoch complete.
Epoch 18/50
Epoch complete.
Epoch 19/50
Epoch complete.
Epoch 20/50
Epoch complete.
Epoch 21/50
Epoch complete.
Epoch 22/50
Epoch complete.
Epoch 23/50
Epoch complete.
Epoch 24/50
Epoch complete.
Epoch 25/50
Epoch complete.
Epoch 26/50
Epoch complete.
Epoch 27/50
Epoch complete.
Epoch 28/50
Epoch complete.
Epoch 29/50
Epoch complete.
Epoch 30/50
Epoch complete.
Epoch 31/50
Epoch complete.
Epoch 32/50
Epoch complete.
Epoch 33/50
Epoch complete.
Epoch 34/50
Epoch complete.
Epoch 35/50
Epoch complete.
Epoch 36/50
Epoch complete.
E

In [29]:
results = []
X1_test=features["X3_test"]
for i in range(X1_test.shape[0]):
    Xi_test = (ctypes.c_int * FEATURES)(*X1_test[i])
    class_sums = [lib.tm_score(tm, Xi_test) for tm in tmachines]
    predicted_label = int(np.argmax(class_sums))
    results.append([i] + class_sums + [predicted_label])

In [30]:
# Define CSV filename.
import csv
csv_filename = "cifar2_hog_C200_CL2_T30_E50.csv"

# Write the results to a CSV file.
with open(csv_filename, "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    # Write header: sample index, one column per class vote sum, and predicted label.
    header = ["sample_index"] + [f"class{cls}_sum" for cls in range(NUM_CLASSES)] + ["predicted_label"]
    writer.writerow(header)
    writer.writerows(results)

print(f"Predictions saved to {csv_filename}")

Predictions saved to cifar2_hog_C200_CL2_T30_E50.csv


In [31]:
import pandas as pd
import numpy as np

csv_files = [
    # "cifar2_3x3_C200_CL2_T30_E50.csv",
    # "cifar2_4x4_C200_CL2_T30_E50.csv",
    "cifar2_hog_C200_CL2_T30_E50.csv",
    # "cifar2_thresh_C200_CL2_T30_E50.csv"
]


dfs = [pd.read_csv(f) for f in csv_files]



class_sums = [df[[f"class{i}_sum" for i in range(2)]].values for df in dfs]


total_class_votes = np.sum(class_sums, axis=0)


overall_preds = np.argmax(total_class_votes, axis=1)


accuracy = np.mean(overall_preds == y_test)
print(f"Overall Accuracy: {accuracy * 100:.2f}%")

Overall Accuracy: 85.33%


In [32]:
lib.tm_save.argtypes = [ctypes.c_void_p, ctypes.c_char_p]
lib.tm_save.restype  = ctypes.c_int

# # Save each TM to its own file:

# for cls, tm in enumerate(tmachines):
#     fname = f"tm_class{cls}.bin".encode('ascii')
#     ret = lib.tm_save(tm, fname)
#     if ret != 0:
#         raise RuntimeError(f"tm_save failed for class {cls} (err {ret})")
# print("All TMs saved.")

# Save each TM to a directory:
save_dir=r"D:\Tsetlin Machines\REDRESS Training\CIFAR2 Model\HOG"
import os
os.makedirs(save_dir, exist_ok=True)

for cls, tm in enumerate(tmachines):
    fname = os.path.join(save_dir, f"tm_class{cls}.bin").encode('ascii')
    ret = lib.tm_save(tm, fname)
    if ret != 0:
        raise RuntimeError(f"tm_save failed for class {cls} (err {ret})")

# X4

In [15]:
import ctypes
import numpy as np

feature=features["X4_train"]
labels = np.asarray(y_train)

NUM_CLASSES = 2
FEATURES = 1025

NUM_EPOCHS = 50
s = 7.5  #float


# Load the shared library
lib = ctypes.CDLL('./DLLs/tm_cifar2_thresh.dll')  # Adjust path if needed

lib.CreateTsetlinMachine.restype = ctypes.c_void_p
lib.CreateTsetlinMachine.argtypes = []

# tm_update(TM_ptr, Xi_ptr, target, s)
lib.tm_update.restype = None
lib.tm_update.argtypes = [
    ctypes.c_void_p,
    ctypes.POINTER(ctypes.c_int),
    ctypes.c_int,
    ctypes.c_float
]

# tm_score(TM_ptr, Xi_ptr)
lib.tm_score.restype = ctypes.c_int
lib.tm_score.argtypes = [
    ctypes.c_void_p,
    ctypes.POINTER(ctypes.c_int)
]

# Optional: expose additional utilities
lib.tm_show_value.restype = ctypes.c_uint
lib.tm_show_value.argtypes = [ctypes.c_void_p, ctypes.c_int]

lib.tm_get_state.restype = ctypes.c_int
lib.tm_get_state.argtypes = [ctypes.c_void_p, ctypes.c_int, ctypes.c_int, ctypes.c_int]

# === Initialize Tsetlin Machines ===
# One binary TM per class for multiclass classification
tmachines = [lib.CreateTsetlinMachine() for _ in range(NUM_CLASSES)]

# === Training Loop ===
for epoch in range(NUM_EPOCHS):
    print(f"Epoch {epoch+1}/{NUM_EPOCHS}")
    for Xi_row, true_label in zip(feature, labels):
        # Convert feature vector to ctypes array
        Xi_ctypes = (ctypes.c_int * FEATURES)(*Xi_row.tolist())
        # Update each class-specific TM
        for cls in range(NUM_CLASSES):
            target = 1 if cls == true_label else 0
            lib.tm_update(tmachines[cls], Xi_ctypes, target, ctypes.c_float(s))
    print("Epoch complete.")

# === Prediction on a sample ===
sample_idx = 0  # adjust as needed
Xi_test = (ctypes.c_int * FEATURES)(*feature[sample_idx].tolist())
class_sums = [lib.tm_score(tm, Xi_test) for tm in tmachines]
predicted_class = int(np.argmax(class_sums))

print("Class vote sums:")
for cls, vote in enumerate(class_sums):
    print(f"  Class {cls}: {vote}")
print(f"Predicted class for sample {sample_idx}: {predicted_class}")


Epoch 1/50
Epoch complete.
Epoch 2/50
Epoch complete.
Epoch 3/50
Epoch complete.
Epoch 4/50
Epoch complete.
Epoch 5/50
Epoch complete.
Epoch 6/50
Epoch complete.
Epoch 7/50
Epoch complete.
Epoch 8/50
Epoch complete.
Epoch 9/50
Epoch complete.
Epoch 10/50
Epoch complete.
Epoch 11/50
Epoch complete.
Epoch 12/50
Epoch complete.
Epoch 13/50
Epoch complete.
Epoch 14/50
Epoch complete.
Epoch 15/50
Epoch complete.
Epoch 16/50
Epoch complete.
Epoch 17/50
Epoch complete.
Epoch 18/50
Epoch complete.
Epoch 19/50
Epoch complete.
Epoch 20/50
Epoch complete.
Epoch 21/50
Epoch complete.
Epoch 22/50
Epoch complete.
Epoch 23/50
Epoch complete.
Epoch 24/50
Epoch complete.
Epoch 25/50
Epoch complete.
Epoch 26/50
Epoch complete.
Epoch 27/50
Epoch complete.
Epoch 28/50
Epoch complete.
Epoch 29/50
Epoch complete.
Epoch 30/50
Epoch complete.
Epoch 31/50
Epoch complete.
Epoch 32/50
Epoch complete.
Epoch 33/50
Epoch complete.
Epoch 34/50
Epoch complete.
Epoch 35/50
Epoch complete.
Epoch 36/50
Epoch complete.
E

In [16]:
X4_test

array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 0],
       [1, 1, 0, ..., 0, 1, 0],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 0],
       [0, 1, 1, ..., 1, 0, 1]], shape=(10000, 1025), dtype=uint8)

In [17]:
results = []
X1_test=features["X4_test"]
for i in range(X1_test.shape[0]):
    Xi_test = (ctypes.c_int * FEATURES)(*X1_test[i])
    class_sums = [lib.tm_score(tm, Xi_test) for tm in tmachines]
    predicted_label = int(np.argmax(class_sums))
    results.append([i] + class_sums + [predicted_label])

In [18]:
# Define CSV filename.
import csv
csv_filename = "cifar2_thresh_C200_CL2_T30_E50.csv"

# Write the results to a CSV file.
with open(csv_filename, "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    # Write header: sample index, one column per class vote sum, and predicted label.
    header = ["sample_index"] + [f"class{cls}_sum" for cls in range(NUM_CLASSES)] + ["predicted_label"]
    writer.writerow(header)
    writer.writerows(results)

print(f"Predictions saved to {csv_filename}")

Predictions saved to cifar2_thresh_C200_CL2_T30_E50.csv


In [19]:
import pandas as pd
import numpy as np

csv_files = [
    # "cifar2_3x3_C200_CL2_T30_E50.csv",
    # "cifar2_4x4_C200_CL2_T30_E50.csv",
    # "cifar2_hog_C200_CL2_T30_E50.csv",
    "cifar2_thresh_C200_CL2_T30_E50.csv"
]


dfs = [pd.read_csv(f) for f in csv_files]



class_sums = [df[[f"class{i}_sum" for i in range(2)]].values for df in dfs]


total_class_votes = np.sum(class_sums, axis=0)


overall_preds = np.argmax(total_class_votes, axis=1)


accuracy = np.mean(overall_preds == y_test)
print(f"Overall Accuracy: {accuracy * 100:.2f}%")

Overall Accuracy: 79.69%


In [20]:
lib.tm_save.argtypes = [ctypes.c_void_p, ctypes.c_char_p]
lib.tm_save.restype  = ctypes.c_int

# # Save each TM to its own file:

# for cls, tm in enumerate(tmachines):
#     fname = f"tm_class{cls}.bin".encode('ascii')
#     ret = lib.tm_save(tm, fname)
#     if ret != 0:
#         raise RuntimeError(f"tm_save failed for class {cls} (err {ret})")
# print("All TMs saved.")

# Save each TM to a directory:
save_dir=r"D:\Tsetlin Machines\REDRESS Training\CIFAR2 Model\Threshold"
import os
os.makedirs(save_dir, exist_ok=True)

for cls, tm in enumerate(tmachines):
    fname = os.path.join(save_dir, f"tm_class{cls}.bin").encode('ascii')
    ret = lib.tm_save(tm, fname)
    if ret != 0:
        raise RuntimeError(f"tm_save failed for class {cls} (err {ret})")

# Final Composite Reading

In [22]:
csv_files = [
    "cifar2_3x3_C200_CL2_T30_E50.csv",
    "cifar2_4x4_C200_CL2_T30_E50.csv",
    "cifar2_hog_C200_CL2_T30_E50.csv",
    "cifar2_thresh_C200_CL2_T30_E50.csv"
]


dfs = [pd.read_csv(f) for f in csv_files]



class_sums = [df[[f"class{i}_sum" for i in range(2)]].values for df in dfs]


total_class_votes = np.sum(class_sums, axis=0)


overall_preds = np.argmax(total_class_votes, axis=1)


accuracy = np.mean(overall_preds == y_test)
print(f"Overall Accuracy: {accuracy * 100:.2f}%")

Overall Accuracy: 89.22%
